In [1]:
from detectionmodel2 import Processor
from augmentation import (FootageAugmenter, FootagePreprocessor)
import cv2
import os
import pandas as pd
import numpy as np
from pathlib import Path
#from IPython.display import Image, display

In [2]:
def get_subdirectories(path):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

In [3]:
def get_filenames(directory):
    dir_path = Path(directory)
    return [f.name for f in dir_path.iterdir() if f.is_file()]

In [11]:
def get_list_of_frames(video_path):
    cap = cv2.VideoCapture(str(video_path))
    list_of_frames = []

    if not cap.isOpened():
        print("Cannot open video")
        exit()

    while True:
        success, frame = cap.read()

        if not success:
            break
            
        list_of_frames.append(frame)

    cap.release()

    return list_of_frames

In [4]:
def video_fall_detection(list_of_frames, model):

    # Define the codec and create VideoWriter object
    #     fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #     out = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(list_of_frames[0].shape[0]), int(list_of_frames[0].shape[1])))

    # Create an instance of Processor
    processor = Processor(model)

    #Create an instance of Footage Preprocessor
    footage_preprocessor = FootagePreprocessor(list_of_frames)    
    
    #Preprocess Images
    preprocessed_frames = footage_preprocessor.preprocess_footage()

    #Augment Images
    for frame in preprocessed_frames:
        # Process the frame using the Processor class
        frame = processor.process(frame)

        # Write the processed frame into the output video
    #         out.write(frame)

    # Display the processed frame
    #         cv2.imshow("Fall Detection", frame)
    #         if cv2.waitKey(1) & 0xFF == ord('q'):
    #             break

    # (DONT NEED?) cap.release()
    #     out.release()
    #     cv2.destroyAllWindows()

    # Return the statistics using the output_stats method from the Processor instance
    return processor.output_stats()

In [ ]:
#COPY PASTE AND REPLACE 2 LINES BELOW

#BRIGHTNESS
for brightness_val in range(50,200,20):
    updated_frames = footage_augmenter.augment_brightness(target_brightness=brightness_val)

#CONNECTION ERROR (FRAME LAG)
for lag_dur in range(1,8):
    updated_frames = footage_augmenter.mimic_frame_lag(lag_probability=0.1,lag_duration=lag_dur)

#CONNECTION ERROR (FRAME LOSS)
for max_skips in range(1,8):
    updated_frames = footage_augmenter.mimic_loss_of_frames(loss_probability=0.05,max_consecutive_skips=max_skips)

#CONNECTION ERROR (INTERMITTENT NOISE IN FRAMES)
for noise_prob in range(1,20,2):
    updated_frames = footage_augmenter.mimic_connection_error_noise(noise_probability=noise_prob/100)

#IMAGE BLUR
for kernel_size in range(1,16,2):
    updated_frames = footage_augmenter.adjust_image_blur(kernel_size=(kernel_size,kernel_size))

#IMAGE ROTATION
for angle in range(10,360,20):
    updated_frames = footage_augmenter.adjust_camera_rotation(angle=angle)

#CONSISTENT IMAGE NOISE
for noise_stdev in range(0,30,5):
    updated_frames = footage_augmenter.add_image_noise(noise_mean=10, noise_stdev=noise_stdev)

In [13]:
def get_test_results(model,video_dir,fall_patterns_dir):
    complete_test_results = []
    for fall_pattern_path in fall_patterns_dir:
        print(fall_pattern_path)
        fall_pattern_full_path = Path(video_dir,fall_pattern_path)
        cam_angles_paths = get_filenames(fall_pattern_full_path)
        for cam_angles_path in cam_angles_paths:
            print(cam_angles_path)
            if cam_angles_path == ".DS_Store":
                continue
            cam_angle_full_path = Path(fall_pattern_full_path,cam_angles_path)
            list_of_frames = get_list_of_frames(cam_angle_full_path)
            #base case
            test_result = video_fall_detection(list_of_frames,model)
            test_result['Angle'] = cam_angles_path[:-4]
            test_result['Fall Type'] = fall_pattern_path
            test_result['Metric'] = ''
            test_result['MetricVal'] = '' 
            complete_test_results.append(test_result)
            footage_augmenter = FootageAugmenter(list_of_frames)
            #updated_frames = list_of_frames
            #updated_frames -> take from above for various augmentations (copy 2 lines). eg. 
            for noise_stdev in range(0,15,5):
                updated_frames = footage_augmenter.add_image_noise(noise_mean=10, noise_stdev=noise_stdev)
                test_result = video_fall_detection(updated_frames,model)
                test_result['Angle'] = cam_angles_path[:-4]
                test_result['Fall Type'] = fall_pattern_path
                test_result['Metric'] = 'image_noise' #change according to what metric is
                test_result['MetricVal'] = noise_stdev #change according to what for loop is running with
                complete_test_results.append(test_result)
    df_results = pd.DataFrame(complete_test_results)
    df_results['model'] = model
    return df_results

In [6]:
video_dir = Path(os.getcwd(),'./Videos')

In [7]:
fall_patterns_dir = ['chute04']

In [8]:
models = ['yolov8s-pose.pt']

In [14]:
complete_df_results = pd.DataFrame()
backup_counter =  0
for model in models:
    print(model)
    df_result = get_test_results(model,video_dir,fall_patterns_dir)
    df_result.to_pickle(f'df_result_{backup_counter}.pkl')
    complete_df_results = pd.concat([complete_df_results,df_result]).reset_index(drop=True)
    backup_counter+=1
complete_df_results.to_pickle(f'complete_results_df.pkl')

yolov8s-pose.pt
chute04
.DS_Store
Baseline_Brendan-1.mov


/Users/brendanlim/Documents/SchoolStuff/Y4S1/Capstone/repo/MedWatch/model_tester/detectionmodel2.py:82: RuntimeWarning: invalid value encountered in double_scalars
  cos_theta = center_line[1] / math.sqrt(center_line[0] ** 2 + center_line[1] ** 2)
/Users/brendanlim/Documents/SchoolStuff/Y4S1/Capstone/repo/MedWatch/model_tester/detectionmodel2.py:82: RuntimeWarning: invalid value encountered in double_scalars
  cos_theta = center_line[1] / math.sqrt(center_line[0] ** 2 + center_line[1] ** 2)
/Users/brendanlim/Documents/SchoolStuff/Y4S1/Capstone/repo/MedWatch/model_tester/detectionmodel2.py:82: RuntimeWarning: invalid value encountered in double_scalars
  cos_theta = center_line[1] / math.sqrt(center_line[0] ** 2 + center_line[1] ** 2)


In [15]:
complete_df_results

,Human Detection,Fall Detection,Human Detection Frame Count,Fall Detection Frame Count,Total Frame Count,Angle,Fall Type,Metric,MetricVal,model
0,True,True,450,129,457,Baseline_Brendan-1,chute04,,,yolov8s-pose.pt
1,True,True,448,129,457,Baseline_Brendan-1,chute04,image_noise,0,yolov8s-pose.pt
2,True,True,332,58,457,Baseline_Brendan-1,chute04,image_noise,5,yolov8s-pose.pt
3,True,True,455,391,457,Baseline_Brendan-1,chute04,image_noise,10,yolov8s-pose.pt
